## Write to Multiple Verilog Files

In [1]:
for i in range(4,65):
    if i%4==0:
        with open('source/cskipa_{}bit.v'.format(i), 'w') as f:
            f.write('''`include "CSkip.v"

module CSkipA_{}bit
    #(parameter WIDTH = {})
  (
      input [WIDTH-1:0] i_add_term1,
      input [WIDTH-1:0] i_add_term2,
      output [WIDTH-1:0]  sum,
      output cout
   );
  
  wire [WIDTH/4-1:0]     w_cout;
  
  wire [WIDTH-1:0]     w_SUM_0;
  wire [WIDTH-1:0]     w_SUM_1;
  
  wire cout0;

  RCA4 rca_inst(.o_result(sum[3:0]), .cout(cout0), .i_add_term1(i_add_term1[3:0]), .i_add_term2(i_add_term2[3:0]), .cin(1'b0));
  SkipLogic4 skip0(.cin_next(w_cout[0]), .i_add_term1(i_add_term1[3:0]), .i_add_term2(i_add_term2[3:0]), .cin(1'b0), .cout(cout0));

  genvar             ii;
  generate 
      for (ii=1; ii<WIDTH/4; ii=ii+1) 
      begin
          CSkip4 cskip4_inst(sum[(ii+1)*4-1:ii*4], w_cout[ii], i_add_term1[(ii+1)*4-1:ii*4],i_add_term2[(ii+1)*4-1:ii*4],w_cout[ii-1]);  
          
       end
  endgenerate

    assign cout=w_cout[WIDTH/4-1];

endmodule'''.format(i,i))
    

In [2]:
import math
for i in range(4,65):
    if i%4!=0:
        with open('source/cskipa_{}bit.v'.format(i), 'w') as f:
            f.write('''`include "CSkip.v"

module CSkipA_{}bit
    #(parameter WIDTH = {})
  (
      input [WIDTH-1:0] i_add_term1,
      input [WIDTH-1:0] i_add_term2,
      output [WIDTH-1:0]  sum,
      output cout
   );
  
  wire [WIDTH/4+1:0]     w_cout;
  
  assign w_cout[0]=1'b0;

  genvar             ii;
  generate 
      for (ii=0; ii<WIDTH/4; ii=ii+1) 
      begin
          CSkip4 cskip4_inst(sum[(ii+1)*4-1:ii*4], w_cout[ii+1], i_add_term1[(ii+1)*4-1:ii*4],i_add_term2[(ii+1)*4-1:ii*4],w_cout[ii]);  
          
       end
  endgenerate
  
  CSkip{} cskip{}_inst(sum[WIDTH-1:WIDTH-WIDTH%4],w_cout[WIDTH/4+1],i_add_term1[WIDTH-1:WIDTH-WIDTH%4],i_add_term2[WIDTH-1:WIDTH-WIDTH%4],w_cout[WIDTH/4]);
  
  assign cout=w_cout[WIDTH/4+1];
endmodule'''.format(i,i,i%4,i%4))
    

## Write Bash Script

In [3]:
with open('CSkipA_synthesis.sh','w') as f:
    f.write('#!/bin/bash\n')
    for i in range(4,65):
        f.write('qflow synthesize CSkipA_{}bit\n'.format(i))

## End